In [ ]:
import sqlite3
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langgraph.checkpoint.sqlite import SqliteSaver

llm = init_chat_model("openai:gpt-4o-mini")

conn = sqlite3.connect(
    "memory.db",
    check_same_thread=False,
)

In [ ]:
class State(MessagesState):
    custom_stuff: str


graph_builder = StateGraph(State)

In [ ]:
@tool
def get_weather(city: str):
    """Gets weather in city"""
    return f"The weather in {city} is sunny."


llm_with_tools = llm.bind_tools(tools=[get_weather])


def chatbot(state: State):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

In [ ]:
tool_node = ToolNode(
    tools=[get_weather],
)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)


graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")

graph = graph_builder.compile(
    checkpointer=SqliteSaver(conn),
)

In [ ]:
async for event in graph.astream(
    {
        "messages": [
            {
                "role": "user",
                "content": "what is the weather in berlin, budapest and bratislava.",
            },
        ]
    },
    stream_mode="updates",
    # config={
    #     "configurable": {
    #         "thread_id": "2",
    #     },
    # },
):
    print(event)

In [ ]:
for state in graph.get_state_history(
    {
        "configurable": {
            "thread_id": "2",
        },
    }
):
    print(state.next)